## Task 2 - Complex Amplitudes
**Input**: A list or array of four complex amplitudes `[a0, a1, a2, a3]` that define the desired two-qubit state.
   - Ensure that the state is normalized:
     $|a_0|^2 + |a_1|^2 + |a_2|^2 + |a_3|^2 = 1$.
   - If the input is not normalized, include a normalization step.

**Output**: A representation of the two-qubit quantum state vector, for example as a NumPy array:
     $\ket{\psi} = a_0\ket{00} + a_1\ket{01} + a_2\ket{10} + a_3\ket{11}.$
 
 - Do not use quantum-specific state preparation functions from libraries.

**Testing**:
   - Write unit tests that check:
     - Normalization is enforced.
     - The output vector has the correct dimension (4 for two qubits).

**Stretch Goal**: Generalize the implementation to support a three-qubit state given 8 amplitudes.

### Solution
---
The following solution is already generalized to $n$ qubits. The number of amplitudes must be $2^n$. I think this solution is pretty self-explanatory, you just take an array of complex numbers, normalize them if needed, and check the dimensionality.

In [3]:
import numpy as np

def prepare_state(amplitudes):
    state = np.array(amplitudes, dtype=np.complex128)

    # Normalize if sum is not 1
    norm = np.linalg.norm(state)
    if not np.isclose(norm, 1.0):
        state = state / norm

    # Check dimension (must be 2^n)
    n_qubits = int(np.log2(len(state)))
    if 2**n_qubits != len(state):
        raise ValueError("Number of amplitudes must be 2^n for n qubits")
    
    return state

The unit tests are provided below.

In [4]:
import unittest

class TestPrepareState(unittest.TestCase):
    def test_normalization(self):
        amps = [0, 1+1j, 0, 0] # (1 + 1i)|01>  -> ((1 + 1i) / sqrt(2)) |01>
        state = prepare_state(amps)
        self.assertTrue(np.isclose(np.linalg.norm(state), 1.0))
    
    def test_dimension(self):
        amps = [1, 0, 0, 0] # |00>
        state = prepare_state(amps)
        self.assertEqual(state.shape, (4,))
    
    def test_invalid_dimension(self):
        with self.assertRaises(ValueError):
            prepare_state([1, 0, 0])  # not 2^n

unittest.main(argv=[''], verbosity=2, exit=False)

test_dimension (__main__.TestPrepareState.test_dimension) ... ok
test_invalid_dimension (__main__.TestPrepareState.test_invalid_dimension) ... ok
test_normalization (__main__.TestPrepareState.test_normalization) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK


#### Extended solution
Since I've tried tackling error correction in task 3, I've put together a quick and dirty library called *qlib* to make my life easier. You can check out the source in the qlib folder, and you can also find it on my github under the name [qipy](https://github.com/vladimirdabic/qipy).

The library focues mainly on density matrices rather than state vectors. It supports two types of simulation: dense and sparse. In dense simulation, it represents the density matrix as a numpy matrix, and performs direct operations on it. In sparse simulation, it represents the density matrix as a list of terms with coefficients. These terms represent entries into the density matrix.

**NOTE**: The sparse simulation stores only *non-zero* density matrix elements. For states with few non-zero terms, this can significantly reduce memory usage compared to storing the full $2^n \times 2^n$ matrix. However, as the number of terms grows, the sparse representation becomes less efficient than dense arrays.

The following examples use the dense simulation. Switching between dense and sparse is not too difficult, as all important functions and classes share the same signature. A state is initialized similarly to the solution of task 2:

In [5]:
import qlib.dense as qld
import qlib as ql

two_qubit_state = qld.State(amplitudes=[1, 0, 0, 1], latex_symbol=r"\rho_{\Phi^+}")  # \phi^+ bell state
three_qubit_state = qld.State(num_qubits=3, latex_symbol=r"\rho_{\text{three}}") # |000> state

display(two_qubit_state, three_qubit_state)

\rho_{\Phi^+} = \frac{1}{2}|00\rangle\langle 00| + \frac{1}{2}|00\rangle\langle 11| + \frac{1}{2}|11\rangle\langle 00| + \frac{1}{2}|11\rangle\langle 11|

\rho_{\text{three}} = |000\rangle\langle 000|

As you can see, a $\LaTeX$ representation is supported, which is quite nice :)

Basic circuit functionality is supported via the Circuit class:

In [7]:
import qlib.dense as qld
import qlib as ql
from IPython.display import Latex

bell_circuit = qld.Circuit(num_qubits=2)
bell_circuit.add_gate(ql.ops.H, qubit=0)
bell_circuit.CNOT(control_qubit=0, target_qubit=1)

state_00 = qld.State(amplitudes=[1, 0, 0, 0])
state_01 = qld.State(amplitudes=[0, 1, 0, 0])

final1 = bell_circuit.run(initial_state=state_00) # produces \phi^+ state
final2 = bell_circuit.run(initial_state=state_01) # produces \psi^+ state

final1.latex_symbol=r"\rho_{\Phi^+}"
final2.latex_symbol=r"\rho_{\Psi^+}"

display(Latex("$$" + state_00.to_latex() + r" \, \to \, " + final1.to_latex() + "$$"))
display(Latex("$$" + state_01.to_latex() + r" \, \to \," + final2.to_latex() + "$$"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Basic noise modelling is also supported via the NoiseModel abstract class:

In [8]:
import qlib as ql
import qlib.dense as qld

# Assumes a single qubit system
class BitFlipChannel(qld.NoiseModel):
    def __init__(self, p: int):
        super().__init__()
        self.p = p

    def apply(self, state: qld.State):
        rho = state.matrix
        rho_noisy = (1 - self.p) * rho + self.p * (ql.ops.X @ rho @ ql.ops.X)
        state.matrix = rho_noisy

circuit = qld.Circuit(num_qubits=1)
circuit.add_gate(ql.ops.X, qubit=0, noise_model=BitFlipChannel(0.2))   # Noisy X gate

state = qld.State(num_qubits=1)
final = circuit.run(initial_state=state)

final

\rho = \frac{1}{5}|0\rangle\langle 0| + \frac{4}{5}|1\rangle\langle 1|

Below is an example of measuring the output of the Bell circuit with multiple shots using the sparse simulation.

In [9]:
import qlib as ql
import qlib.sparse as qls

circuit = qls.Circuit(num_qubits=2)
circuit.add_gate(ql.ops.H, qubit=0)
circuit.CNOT(control_qubit=0, target_qubit=1)

state = qls.State(num_qubits=2)
results = {}

# run_shots yields the final state for each run
for final in circuit.run_shots(initial_state=state, shots=100):
    meas_res = final.measure(qubits=[0, 1])
    res_str = ''.join(str(i) for i in meas_res)

    results[res_str] = results.get(res_str, 0) + 1

results

{'00': 57, '11': 43}